In [2]:
import tensorflow as tf
from gensim import models
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import StratifiedKFold

In [21]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = vectorizer(np.array([[s] for s in train_x])).numpy()    
    val_x = vectorizer(np.array([[s] for s in val_x])).numpy()
    test_x = vectorizer(np.array([[s] for s in test_x])).numpy()

    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, 
                                       output_dim = emb_dim, 
                                       embeddings_initializer=keras.initializers.Constant(matrix),
                                       trainable=True)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.LSTM(32, input_dim=(None, emb_dim), 
                                         return_sequences=False, activation='tanh', 
                                         dropout=0.3))(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "lstm_models/{itr:04}".format(itr=cnt)
    checkpoint_path = model_path + "-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    model.compile(tf.keras.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘lstm_models/0001’: No such file or directory
Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6729 - accuracy: 0.5689 - auc: 0.6007
Epoch 00001: val_auc improved from -inf to 0.68713, saving model to lstm_models/0001-0001.ckpt
63/63 [==============================] - 14s 219ms/step - loss: 0.6729 - accuracy: 0.5689 - auc: 0.6007 - val_loss: 0.6365 - val_accuracy: 0.6240 - val_auc: 0.6871
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.3793 - accuracy: 0.8506 - auc: 0.9230
Epoch 00002: val_auc improved from 0.68713 to 0.87542, saving model to lstm_models/0001-0002.ckpt
63/63 [==============================] - 12s 195ms/step - loss: 0.3793 - accuracy: 0.8506 - auc: 0.9230 - val_loss: 0.4411 - val_accuracy: 0.8040 - val_auc: 0.8754
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.0799 - accuracy: 0.9795 - auc: 0.9966
Epoch 00003: val_auc improved from 0.87542 to 0.88017, saving model to lstm_mo

In [22]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.6327536582946778 +/- 0.09062245368069972
test avg acc:  0.7998000025749207 +/- 0.01570223105062399
test avg auc:  0.8621880054473877 +/- 0.014906734082370676


val avg loss:  0.5842467606067657 +/- 0.0687760712357067
val avg acc:  0.8076000094413758 +/- 0.013290602153058276
val avg auc:  0.8740259647369385 +/- 0.009767909844085145


train avg loss:  0.01478144726715982 +/- 0.011762669993772167
train avg acc:  0.9979249954223632 +/- 0.0017113562754474138
train avg auc:  0.999770975112915 +/- 0.00016101809099796976
